In [2]:
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from mtcnn import MTCNN
from PIL import Image

In [3]:
feature_list = np.array(pickle.load(open('embedding.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))

In [6]:
print(f"{filenames[1:10]} {feature_list[1:10]}")

['actors\\Actors_faces\\Aamir_Khan1.jpg', 'actors\\Actors_faces\\Aamir_Khan101.jpg', 'actors\\Actors_faces\\Aamir_Khan103.jpg', 'actors\\Actors_faces\\Aamir_Khan105.jpg', 'actors\\Actors_faces\\Aamir_Khan107.jpg', 'actors\\Actors_faces\\Aamir_Khan109.jpg', 'actors\\Actors_faces\\Aamir_Khan11.jpg', 'actors\\Actors_faces\\Aamir_Khan111.jpg', 'actors\\Actors_faces\\Aamir_Khan113.jpg'] [[0.0000000e+00 7.5507373e-02 0.0000000e+00 ... 2.2958543e+00
  4.0392363e-03 3.7423279e+00]
 [0.0000000e+00 6.3186491e-01 0.0000000e+00 ... 3.0350797e+00
  0.0000000e+00 6.9678955e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 8.9448032e+00
  7.3518120e-02 5.8362813e+00]
 ...
 [5.0303861e-02 2.1379738e+00 0.0000000e+00 ... 1.7698953e+00
  6.7080361e-01 2.2449567e+00]
 [7.0421994e-02 1.8419120e-01 0.0000000e+00 ... 7.8910761e+00
  1.1021263e-03 3.1068077e+00]
 [0.0000000e+00 1.8006738e-02 0.0000000e+00 ... 1.2053572e+01
  8.8220453e-01 1.8553628e-01]]


In [7]:
model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

In [8]:
detector = MTCNN()

In [12]:
s_img = cv2.imread('D:/python_ml/face/sample/4.jpg')
results = detector.detect_faces(s_img)
results

[{'box': [375, 57, 115, 151],
  'confidence': 0.9931060671806335,
  'keypoints': {'left_eye': (400, 115),
   'right_eye': (455, 113),
   'nose': (423, 147),
   'mouth_left': (404, 168),
   'mouth_right': (456, 166)}}]

In [13]:
x,y,width,height = results[0]['box']
face = s_img[y:y+height,x:x+width]

In [14]:
image = Image.fromarray(face)
image = image.resize((224,224))
face_array = np.asarray(image)
face_array = face_array.astype('float32')
expanded_img = np.expand_dims(face_array,axis=0)
preprocessed_img = preprocess_input(expanded_img)
result = model.predict(preprocessed_img).flatten()
result

array([0.01244888, 0.4249769 , 3.3131335 , ..., 0.        , 1.5588728 ,
       2.3736274 ], dtype=float32)

In [15]:
similarity = []
for i in range(len(feature_list)):
    similarity.append(cosine_similarity(result.reshape(1,-1),feature_list[i].reshape(1,-1))[0][0])

index_pos = sorted(list(enumerate(similarity)),reverse=True,key=lambda x:x[1])[0][0]

temp_img = cv2.imread(filenames[index_pos])
cv2.imshow('output',temp_img)
cv2.waitKey(0)

-1